<a href="https://colab.research.google.com/github/suryaa2910/Translator_Tamil_Transformer/blob/main/Tamil_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Remove non alphanumeric characters for simple training

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformer import Transformer # this is the transformer.py file
import torch
import numpy as np

In [ ]:
english_file = "/content/drive/MyDrive/Suryaa_Project/DataSet_Capstone/train.en" # replace this path with appropriate one
kannada_file = "/content/drive/MyDrive/Suryaa_Project/DataSet_Capstone/train.ta"# replace this path with appropriate one


# Generated this by filtering Appendix code

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

kannada_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',
                    'ஃ', 'அ', 'ஆ', 'இ', 'ஈ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஐ', 'ஒ', 'ஓ', 'ஔ',
                    'க', 'கா', 'கி', 'கீ', 'கு', 'கூ', 'கெ', 'கே', 'கை', 'கொ', 'கோ', 'கௌ',
                    'ச', 'சா', 'சி', 'சீ', 'சு', 'சூ', 'செ', 'சே', 'சை', 'சொ', 'சோ', 'சௌ',
                    'ஞ', 'ஞா', 'ஞி', 'ஞீ', 'ஞு', 'ஞூ', 'ஞெ', 'ஞே', 'ஞை', 'ஞொ', 'ஞோ', 'ஞௌ',
                    'ங' , 'ஙா' , 'ஙி' , 'ஙீ' , 'ஙு' , 'ஙூ' , 'ஙெ' , 'ஙே' , 'ஙை' , 'ஙொ' , 'ஙோ' , 'ஙௌ' ,
                    'ட', 'டா', 'டி', 'டீ', 'டு', 'டூ', 'டெ', 'டே', 'டை', 'டொ', 'டோ', 'டௌ',
                    'ண', 'ணா', 'ணி', 'ணீ', 'ணு', 'ணூ', 'ணெ', 'ணே', 'ணை', 'ணொ', 'ணோ', 'ணௌ',
                    'ன' , 'னா' , 'னி' , 'னீ' , 'னு' , 'னூ' , 'னெ' , 'னே' , 'னை' , 'னொ' , 'னோ' , 'னௌ',
                    'த', 'தா', 'தி', 'தீ', 'து', 'தூ', 'தெ', 'தே', 'தை', 'தொ', 'தோ', 'தௌ',
                    'ந', 'நா', 'நி', 'நீ', 'நு', 'நூ', 'நெ', 'நே', 'நை', 'நொ', 'நோ', 'நௌ',
                    'ப', 'பா', 'பி', 'பீ', 'பு', 'பூ', 'பெ', 'பே', 'பை', 'பொ', 'போ', 'பௌ',
                    'ம', 'மா', 'மி', 'மீ', 'மு', 'மூ', 'மெ', 'மே', 'மை', 'மொ', 'மோ', 'மௌ',
                    'ய', 'யா', 'யி', 'யீ', 'யு', 'யூ', 'யெ', 'யே', 'யை', 'யொ', 'யோ', 'யௌ',
                    'ர', 'ரா', 'ரி', 'ரீ', 'ரு', 'ரூ', 'ரெ', 'ரே', 'ரை', 'ரொ', 'ரோ', 'ரௌ',
                    'ல', 'லா', 'லி', 'லீ', 'லு', 'லூ', 'லெ', 'லே', 'லை', 'லொ', 'லோ', 'லௌ',
                    'ள', 'ளா', 'ளி', 'ளீ', 'ளு', 'ளூ', 'ளெ', 'ளே', 'ளை', 'ளொ', 'ளோ', 'ளௌ',
                    'ழ', 'ழா', 'ழி', 'ழீ', 'ழு', 'ழூ', 'ழெ', 'ழே', 'ழை', 'ழொ', 'ழோ', 'ழௌ',
                    'வ', 'வா', 'வி', 'வீ', 'வு', 'வூ', 'வெ', 'வே', 'வை', 'வொ', 'வோ', 'வௌ',
                    'ஷ', 'ஷா', 'ஷி', 'ஷீ', 'ஷு', 'ஷூ', 'ஷெ', 'ஷே', 'ஷை', 'ஷொ', 'ஷோ', 'ஷௌ',
                    'ஸ', 'ஸா', 'ஸி', 'ஸீ', 'ஸு', 'ஸூ', 'ஸெ', 'ஸே', 'ஸை', 'ஸொ', 'ஸோ', 'ஸௌ',
                    'ஹ', 'ஹா', 'ஹி', 'ஹீ', 'ஹு', 'ஹூ', 'ஹெ', 'ஹே', 'ஹை', 'ஹொ', 'ஹோ', 'ஹௌ',
                    'ா', 'ி', 'ீ', 'ு', 'ூ', 'ெ', 'ே', 'ை', 'ொ', 'ோ', 'ௌ', '்', '௕', '௖', '௞', '௣', 'ஂ', 'ஃ',
                    '௦', '௧', '௨', '௩', '௪', '௫', '௬', '௭', '௮', '௯', PADDING_TOKEN, END_TOKEN]

#
#
#
english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
index_to_kannada = {k:v for k,v in enumerate(kannada_vocabulary)}
kannada_to_index = {v:k for k,v in enumerate(kannada_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [ ]:
with open(english_file, 'r',encoding='utf-8') as file:
    english_sentences = file.readlines()
with open(kannada_file, 'r',encoding='utf-8') as file:
    kannada_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
kannada_sentences = kannada_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
kannada_sentences = [sentence.rstrip('\n') for sentence in kannada_sentences]

In [ ]:
english_sentences[:10]

['some 14 months later, the second calf is born.',
 '"senior advocate kapil sibal, who was appearing for chidambaram, said the condition was not justified for a member of parliament and ""he would not run away anywhere."""',
 'this photo was taken then.',
 'so far two rounds of the jwg meeting have been held.',
 'the life of the world is nothing but play and diversion, and the abode of the hereafter is surely better for those who are godwary. do you not exercise your reason?',
 'the maximum temperature will be 35 degrees celsius and the minimum will settle at 27 degree.',
 "i think with all these lives what you're going to find is that your own experience tends to color a good deal which one you pick as closer to the top or closer to the bottom and the idea of a sibling or a child that's no longer involved with the family if your experience with that is very painful, then the chances are that you'll kind of push that one to the bottom.",
 'if they agreed with a point the speaker made, 

In [ ]:
kannada_sentences[:10]

['சுமார் 14 மாதங்கள் கழித்து, இரண்டாம் கன்றை ஈனுகிறது.',
 '‘காா்த்தி சிதம்பரம் எம். பி. யாக உள்ளதால் எங்கும் தப்பிவிட மாட்டாா்’ என்று அவரது சாா்பில் ஆஜரான மூத்த வழக்குரைஞா் கபில் சிபல் வாதாடினாா்.',
 'அதன்போது எடுக்கப்பட்ட புகைப்படம் இது.',
 'இதுவரை இணைப் பணிக் குழு இரண்டுகட்ட பேச்சுவார்த்தை முடிவடைந்துள்ளது.',
 'உலக வாழ்க்கை வீணும் விளையாட்டுமேயன்றி வேறில்லை பயபக்தியுடையவர்களுக்கு நிச்சயமாக மறுமை வீடே மிகவும் மேலானதாகும். நீங்கள் இதைப் புரிந்து கொள்ள வேண்டாமா?',
 'அதிகபட்ச வெப்பநிலை 35 டிகிரி செல்சியஸாகவும் குறைந்தபட்சம் 27 டிகிரி செல்சியஸாகவும் இருக்கும்.',
 'இந்த எல்லா உயிர்களிலும் நீங்கள் கண்டுபிடிக்கப் போவது இதுதான் என்று நான் நினைக்கிறேன் உங்கள் சொந்த அனுபவம் நீங்கள் தேர்ந்தெடுக்கும் ஒரு நல்ல ஒப்பந்தத்தை வண்ணமயமாக்குகிறது மேலே நெருக்கமாக அல்லது கீழே நெருக்கமாக மற்றும் யோசனை ஒரு உடன்பிறப்பு அல்லது இனி சம்பந்தப்படாத குழந்தை உங்கள் அனுபவம் மிகவும் வேதனையாக இருந்தால் குடும்பத்துடன், பின்னர் நீங்கள் அதை கீழே தள்ளும் வாய்ப்புகள் உள்ளன.',
 'பேச்சாளர் சொன்ன ஒரு குறிப்போடு அவர்கள் ஒத்திர

In [ ]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in kannada_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length Kannada: 239.0
97th percentile length English: 216.0


In [ ]:
max_sequence_length = 1000

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(kannada_sentences)):
    kannada_sentence, english_sentence = kannada_sentences[index], english_sentences[index]
    if is_valid_length(kannada_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(kannada_sentence, kannada_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(kannada_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 200000
Number of valid sentences: 63754


In [ ]:
kannada_sentences = [kannada_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [ ]:
kannada_sentences[:3]

['அதன்போது எடுக்கப்பட்ட புகைப்படம் இது.',
 'இதுவரை இணைப் பணிக் குழு இரண்டுகட்ட பேச்சுவார்த்தை முடிவடைந்துள்ளது.',
 'இது சம்பந்தமாக ஒரு வீடியோவையும் வெளியிட்டு உள்ளார்.']

In [ ]:
import torch

d_model = 512
batch_size = 2
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 1000
kn_vocab_size = len(kannada_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          kn_vocab_size,
                          english_to_index,
                          kannada_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [ ]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, kannada_sentences):
        self.english_sentences = english_sentences
        self.kannada_sentences = kannada_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.kannada_sentences[idx]

In [ ]:
dataset = TextDataset(english_sentences, kannada_sentences)

In [ ]:
len(dataset)

63754

In [ ]:
dataset[1]

('so far two rounds of the jwg meeting have been held.',
 'இதுவரை இணைப் பணிக் குழு இரண்டுகட்ட பேச்சுவார்த்தை முடிவடைந்துள்ளது.')

In [ ]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [ ]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('this photo was taken then.', 'so far two rounds of the jwg meeting have been held.'), ('அதன்போது எடுக்கப்பட்ட புகைப்படம் இது.', 'இதுவரை இணைப் பணிக் குழு இரண்டுகட்ட பேச்சுவார்த்தை முடிவடைந்துள்ளது.')]
[('with this, a video was released as well.', 'its not your fault.'), ('இது சம்பந்தமாக ஒரு வீடியோவையும் வெளியிட்டு உள்ளார்.', 'இது உன் பிழையல்ல.')]
[('what is joy about?', 'worship lord shiva.'), ('அப்படி என்ன ஆனந்தம்?', 'சிவபெருமானை அதிகமாக வழிபடுங்கள்.')]
[("that's actually not precisely right.", 'what you were taught in school?'), ('அது சரியானது இல்லை.', 'உனக்குக் கல்லூரியிலே என்ன பாடம் ?')]
[('up top', 'i really like the telugu language.'), ('மேலே உயரே,', 'தமிழ் மொழி எனக்கு ரொம்ப பிடிக்கும்.')]


In [ ]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=kannada_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

Modify mask such that the padding tokens cannot look ahead.
In Encoder, tokens before it should be -1e9 while tokens after it should be -inf.


Note the target mask starts with 2 rows of non masked items: https://github.com/SamLynnEvans/Transformer/blob/master/Beam.py#L55


In [ ]:
# Corrected code


transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10
accumulation_steps = 4  # You can adjust this value
for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        kn_predictions = transformer(eng_batch,
                                    kn_batch,
                                    encoder_self_attention_mask.to(device),
                                    decoder_self_attention_mask.to(device),
                                    decoder_cross_attention_mask.to(device),
                                    enc_start_token=True,  # Adjusted to include start token in encoder
                                    enc_end_token=False,
                                    dec_start_token=True,
                                    dec_end_token=False)   # Adjusted to exclude end token in decoder

        labels = transformer.decoder.sentence_embedding.batch_tokenize(kn_batch, start_token=False, end_token=True)
        loss = criterian(
            kn_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == kannada_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        if (batch_num + 1) % accumulation_steps == 0:
            optim.step()
            optim.zero_grad()
        # optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Tamil Translation: {kn_batch[0]}")
            kn_sentence_predicted = torch.argmax(kn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
                if idx == kannada_to_index[END_TOKEN]:
                    break
                    predicted_sentence += index_to_kannada[idx.item()]
            print(f"Tamil Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions =transformer(eng_batch,
                                    kn_batch,
                                    encoder_self_attention_mask.to(device),
                                    decoder_self_attention_mask.to(device),
                                    decoder_cross_attention_mask.to(device),
                                    enc_start_token=True,  # Adjusted to include start token in encoder
                                    enc_end_token=False,
                                    dec_start_token=True,
                                    dec_end_token=False)   # Adjusted to exclude end token in decoder

                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_kannada[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                    break

            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 1.06 MiB is free. Process 2058 has 14.74 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 84.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Original Code

transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 1

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, kn_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        kn_predictions = transformer(eng_batch,
                                    kn_batch,
                                    encoder_self_attention_mask.to(device),
                                    decoder_self_attention_mask.to(device),
                                    decoder_cross_attention_mask.to(device),
                                    enc_start_token=True,  # Adjusted to include start token in encoder
                                    enc_end_token=False,
                                    dec_start_token=True,
                                    dec_end_token=False)   # Adjusted to exclude end token in decoder

        labels = transformer.decoder.sentence_embedding.batch_tokenize(kn_batch, start_token=False, end_token=True)
        loss = criterian(
            kn_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == kannada_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Kannada Translation: {kn_batch[0]}")
            kn_sentence_predicted = torch.argmax(kn_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
                if idx == kannada_to_index[END_TOKEN]:
                    break
                    predicted_sentence += index_to_kannada[idx.item()]
            print(f"Kannada Prediction: {predicted_sentence}")


            transformer.eval()
            kn_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions =transformer(eng_batch,
                                    kn_batch,
                                    encoder_self_attention_mask.to(device),
                                    decoder_self_attention_mask.to(device),
                                    decoder_cross_attention_mask.to(device),
                                    enc_start_token=True,  # Adjusted to include start token in encoder
                                    enc_end_token=False,
                                    dec_start_token=True,
                                    dec_end_token=False)   # Adjusted to exclude end token in decoder

                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_kannada[next_token_index]
                kn_sentence = (kn_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                    break

            print(f"Evaluation translation (should we go to the mall?) : {kn_sentence}")
            print("-------------------------------------------")

Epoch 0


OutOfMemoryError: CUDA out of memory. Tried to allocate 62.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 9.06 MiB is free. Process 2058 has 14.74 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 107.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Inference

In [ ]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  kn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_kannada[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]

In [ ]:
translation = translate("what should we do when the day starts?")
print(translation)
#ದಿನ ಪ್ರಾರಂಭವಾದಾಗ ನಾವು ಏನು ಮಾಡಬೇಕು?

RuntimeError: The size of tensor a (1001) must match the size of tensor b (1000) at non-singleton dimension 1

In [ ]:
translation = translate("how is this the truth?")
print(translation)
#ಇದು ಹೇಗೆ ಸತ್ಯ

ಹೇಗೆ ಇದು ಹೇಗೆ ಹೇಗೆ?<END>


In [ ]:
translation = translate("the world is a large place with different people")
print(translation)
#ಪ್ರಪಂಚವು ವಿಭಿನ್ನ ಜನರೊಂದಿಗೆ ದೊಡ್ಡ ಸ್ಥಳವಾಗಿದೆ

ಇದರಿಂದ ಮೂಲಕ ಸಂಬಂಧಿಸಿದ ಮೇಲೆ ಮಾಡಿದ್ದಾರೆ<END>


In [ ]:
translation = translate("my name is ajay")
print(translation)
#ನನ್ನ ಹೆಸರು ಅಜಯ್

ನಾನು ಕುಟುಂಬದ ಹೆಸರು<END>


In [ ]:
translation = translate("i cannot stand this smell")
print(translation)
#ನಾನು ಈ ವಾಸನೆಯನ್ನು ಸಹಿಸುವುದಿಲ್ಲ

ನಾನು ಅಂತರ ಸಂಗತಿ ನಾನು ಕೊಡುವುದಿಲ್ಲ<END>


In [ ]:
translation = translate("noodles are the best")
print(translation)

ಯಾವುದೇ ಕಾರಣಗಳು ಇಲ್ಲ<END>


In [ ]:
translation = translate("why care about this?")
print(translation)

ಏಕೆ ಕಾರಣ ಏನು?<END>


This translated pretty well : "What is the reason. Why" without punctuation.

In [ ]:
translation = translate("this is the best thing ever")
print(translation)
# ಇದು ಎಂದೆಂದಿಗೂ ಉತ್ತಮವಾಗಿದೆ

ಇದು ಅತ್ಯಂತ ಹೊರತಾಗಿದೆ<END>


The translation : "This is very unusual"

In [ ]:
translation = translate("i am here")
print(translation)
# ನಾನು ಇಲ್ಲಿದ್ದೇನೆ

ನಾನು ಕೇಳಿದ್ದೇನೆ.<END>


Translation: "I have heard".
This is why word based translator may perform better than character translator. This is actually very good at optimizing the objective of the current transformer even though the translation is off.

In [ ]:
translation = translate("click this")
print(translation)
# ಇದನ್ನು ಕ್ಲಿಕ್ ಮಾಡಿ

ಕ್ಯಾನ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಮಾಡಿ<END>


In [ ]:
translation = translate("where is the mall?")
print(translation)

ಎಲ್ಲಿ ಎಲ್ಲಿ ಎಲ್ಲಿ?<END>


In [ ]:
translation = translate("what should we do?")
print(translation)

ಏನು ಮಾಡಬೇಕು?<END>


This is correct; but it absolutely fumbles on the next one

In [ ]:
translation = translate("today, what should we do")
print(translation)

ಅದನ್ನು ಮಾಡಿದ ಮೇಲೆ ಮಾಡಿದ ಮಾಡಿದ್ದಾರೆ<END>


In [ ]:
translation = translate("why did they activate?")
print(translation)
# ಅವರು ಏಕೆ ಸಕ್ರಿಯಗೊಳಿಸಿದರು?

ಅದರ ಮೇಲೆ ಏಕೆ ಮಾಡಿದ್ದಾರೆ?<END>


In [ ]:
translation = translate("why did they do this?")
print(translation)
# ಅವರು ಇದನ್ನು ಏಕೆ ಮಾಡಿದರು?

ಅವರು ಏಕೆ ಅವರು ಏಕೆ ಮಾಡಿದ್ದಾರೆ?<END>


That turned out well!

In [ ]:
translation = translate("i am well.")
print(translation)
# ನಾನು ಆರಾಮವಾಗಿದ್ದೇನೆ

ನಾನು ನಾನು ಕೊಡುತ್ತೇನೆ.<END>


Translation: "I will give you something"

In [ ]:
translation = translate("whats the word on the street?")
print(translation)

ಇದರ ಬಗ್ಗೆ ಏನು?<END>


Kind of close semantically. Translation is something like: "What is this about"

## Insights

- When training, we can treat every alphabet as a single unit instead of splitting it into it's corresponding parts to preserve meaning. For example, ಮಾ should be 1 unit when comuting a loss. It should not be decomposed into ಮ + ఆ
- Using word-based or BPE based tokenizations may help mitigate (1). Also, we will get valid word (or BPE) units if we do so.
- Make sure the training set has a large variety of sentences that are not just about one topic like "work" and "government"
- Increase the number of encoder / decoder units for better translations. It was set to the minimum of 1 of each unit here.
- Create a translator with a language you understand ideally.

Overall, this model definately learned something. And you can use other languages instead of this kannada language and might see better luck